In [1]:
%matplotlib qt
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors
from matplotlib import cm
from matplotlib.ticker import NullFormatter
from mpl_toolkits.mplot3d import Axes3D

from scipy import stats
from scipy.stats import multivariate_normal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from skimage.transform import resize

import math

import seaborn as sns; sns.set()
np.random.seed(42)

In [2]:
class genere_distributions():
    x_min, x_max = 0., 10.
    y_min, y_max = 0., 10.
    nx, ny = 300, 300
    rstride, cstride = 5, 5
    npts = 10000
    
    def __init__(self, mu, sigma, angle, prob_C):
        self.mu = mu
        self.sigma = sigma
        self.angle = angle
        self.prob_C = prob_C
        
        # Calcul du nombre de points 2D dans chacun des 2 nuages de points. 
        self.n = (self.npts*prob_C).astype(int)
        
        self.genere_dataset()
        self.genere_classif()
        self.genere_pdf()
        

    # Génère deux nuages de points 2D basés sur les 2 vraies distributions étudiées
    def genere_dataset(self):
        # Calcule les matrices de covariance
        self.calcule_mat_cov()
        
        self.X0 = np.random.multivariate_normal(self.mu[0], self.cov[:,:,0], self.n[0])        
        self.X1 = np.random.multivariate_normal(self.mu[1], self.cov[:,:,1], self.n[1])
        self.X = np.r_[self.X0, self.X1]
        self.y = np.hstack((np.zeros(self.n[0]), np.ones(self.n[1])))


    # Calcul des matrices de covariance basées sur les valeurs des sigma et de l'angle de rotation
    def calcule_mat_cov(self):
        self.cov = np.zeros((2, 2, 2))
        for i in range(2):
            # Matrice de rotation
            theta = np.radians(self.angle[i])
            c, s = np.cos(theta), np.sin(theta)
            R = np.array([[c, -s], [s, c]])
            
            # Matrice de covariance sans rotation
            C = np.array([[self.sigma[i, 0]**2, 0.],[0., self.sigma[i, 1]**2]])

            # Matrice de covariance après rotation
            # new_cov = rotation_matrix @ cov @ rotation_matrix.T
            self.cov[:,:,i] = R.dot(C.dot(R.T) ) 


    # Calcul des positions (x,y) d'un maillage régulier couvrant le plan XY
    def genere_grid(self):
        self.xx, self.yy = np.meshgrid(np.linspace(self.x_min, self.x_max, self.nx), 
                                       np.linspace(self.y_min, self.y_max, self.ny))
        self.pos = np.dstack((self.xx, self.yy))
        
            
    # Modèles génératifs pour les deux distributions normales 2D
    # (PDF: probability distribution functions)
    def genere_pdf(self):
        modeles = [None]*2
        
        for i in range(2):
            # Génère modele pour PDF normale 2D
            modeles[i] = multivariate_normal(self.mu[i,:], self.cov[:,:,i]) 

        # Génère PDF individuelles
        pdf0 = self.prob_C[0]*modeles[0].pdf(self.pos)
        pdf1 = self.prob_C[1]*modeles[1].pdf(self.pos)

        # PDF globale
        self.pdf = pdf0 + pdf1

        
    # Estimation des paramètres des gaussiennes. On utilise les données générées avec les vraies distributions.
    def estime_parametres_dataset(self):        
        moy = np.zeros([2,2])
        cov = np.zeros([2,2,2])
        pC  = np.zeros([2,])

        moy[0,:] = np.mean(self.X0,axis=0)
        cov[:,:,0] = np.cov(self.X0.T)
        pC[0] = self.prob_C[0]
        
        moy[1,:] = np.mean(self.X1,axis=0)
        cov[:,:,1] = np.cov(self.X1.T)
        pC[1] = self.prob_C[1]            
        
        return (moy, cov, pC)
        
        
    # Calcul d'une fonction discriminante
    def fonction_discriminante(self, moy, S, pC, X):
        moy = np.reshape(moy.T, [2,1])
        
        S_inv = np.linalg.inv(S)
        det_S = np.linalg.det(S)
               
        w0 = -0.5*math.log(det_S) + math.log(pC)
        
        ww = np.zeros([X.shape[1]])
        for i in range(X.shape[1]):
            dX = np.reshape(X[:,i], [2,1]) - moy
            ww[i] = (dX.T).dot(S_inv.dot(dX))
        
        h = ww + w0
        
        return h    
    
    
    # Fonction générant la fonction disciminante de chaque classe puis les applique afin d'identifier la zone
    # d'influence de chaque classe. 
    def genere_classif(self):
        # Estimation des paramètres des gaussiennes à partir des données
        (moy, cov, pC) = self.estime_parametres_dataset()
        
        # Calcul des positions (x,y) d'un maillage régulier couvrant le plan xy
        self.genere_grid()
        X = np.c_[self.xx.ravel(), self.yy.ravel()]
        X = X.T
        
        # Calcul des fonctions discriminantes 
        h0 = self.fonction_discriminante(moy[0,:], cov[:,:,0], pC[0], X)        
        h1 = self.fonction_discriminante(moy[1,:], cov[:,:,1], pC[1], X)
        
        # Identifie la zone d'influence de chaque nuage de points en prédisant la classe pour chaque 
        # position (x,y) dans le maillage 2D.
        Z = h1>h0   
        Z = Z.reshape(self.xx.shape)
        self.masque = Z
      
    
    
    # Génération de la surface 3D en 2 couleurs identifiant la zone d'influence de chaque nuage    
    def genere_surf_3D(self, ax):

        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='gray', edgecolors='k')       
        a1 = s.__dict__['_original_facecolor']
        b1 = s.__dict__['_facecolors']
        c1 = s.__dict__['_facecolors3d']
        
        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='w', edgecolors='k')
        a2 = s.__dict__['_original_facecolor']
        b2 = s.__dict__['_facecolors']
        c2 = s.__dict__['_facecolors3d']
        
        Lx = int(self.nx/self.rstride)
        Ly = int(self.ny/self.cstride)

        mask = resize(self.masque, (Lx,Ly), order=0)
        indx = np.argwhere(mask)
        idx = indx[:,0]*Lx + indx[:,1]

        a = a1
        b = b1
        c = c1
        for i in idx:
            a[i,:] = a2[i,:]
            b[i,:] = b2[i,:]
            c[i,:] = c2[i,:]
        s.__dict__['_original_facecolor'] = a
        s.__dict__['_facecolors'] = b
        s.__dict__['_facecolors3d'] = c

   

    # Affiche la fonction de distribution globale en 3D avec ses contours en 2D. L'orientation de 
    # la figure peut être ajustée avec la souris; cela affiche la valeur de la variable view = [élévation, azimuth]
    def affiche_PDF_avec_contours(self, contours_remplis=True, affiche_labels=True, affiche_tickmarks=True, 
                                  view=[20., -20.], offset = -0.15, nom_figure=None):

        # Permet l'affichage de l'azimuth et de l'élévation lors de l'ajustement de l'angle de vue avec la souris.
        fig = plt.figure(figsize = (10,10))
        fig.canvas.set_window_title('3D')
        ax = fig.gca(projection='3d')

        # Génération de la surface 3D en 2 couleurs identifiant les zones d'influence de chaque nuage
        self.genere_surf_3D(ax)
    
        # Contours 2D remplis en dessous
        if (contours_remplis==True):
            cset = ax.contourf(self.xx, self.yy, self.pdf, zdir='z', offset=offset, cmap='viridis')
        else:
            cset = ax.contour(self.xx, self.yy, self.pdf, zdir='z', offset=offset, levels = 10, cmap='hot') 
            
        # Affiche frontières entre les zones d'influence
        ax.contour(self.xx, self.yy, self.masque, [0.5], offset=offset, linewidths=2., colors='white') 
  
       

        ax.set_zlim(offset,np.max(self.pdf))

        ax.view_init(view[0], view[1])
        
        if (affiche_labels==True):
            ax.set_ylabel('$x_{2}$', fontsize=18)
            ax.xaxis.set_rotate_label(False)  
            ax.set_xlabel('$x_{1}$', rotation=10, fontsize=18)

        if (affiche_tickmarks==False):
            # Enlève tickmarks
            ax.xaxis.set_major_formatter(NullFormatter())
            ax.yaxis.set_major_formatter(NullFormatter())
            ax.zaxis.set_major_formatter(NullFormatter())
            
        fig.tight_layout()
        
        
        # Sauvegarde de l'image 
        if nom_figure!=None:
            plt.savefig(nom_figure, format="svg")
            
            
        # Affiche azimuth et élévation lorsque l'on change l'angle de vue avec la souris
        def on_click(event):
            print('view= [%.1f , %.1f]' % (ax.elev, ax.azim))   
            
        cid = fig.canvas.mpl_connect('button_release_event', on_click)
        
        
        plt.show()


In [3]:
if __name__=='__main__':
        
    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [4., 4.]
    mu[1,:] = [6., 6.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [1.5, .5]
    sigma[1,:] = [0.5, 0.5]

    angle = np.array([45., 0.]) 

    prob_C = np.array([0.7, 0.3]) 
    

    nom_figure = "Figure_19_41.svg"

    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset    
    # pdf.affiche_PDF_avec_contours(offset = -0.15)
    
    # Affichage et sauvegarde des meilleurs résultats
    pdf.affiche_PDF_avec_contours(affiche_tickmarks=False, nom_figure=nom_figure)
    

In [4]:
    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [4., 4.]
    mu[1,:] = [6., 7.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [0.5, 0.5]
    sigma[1,:] = [1.2, 0.7]

    angle = np.array([-30., 0.])  

    prob_C = np.array([0.3, 0.7]) 


    nom_figure = "Figure_20(1)_41.svg"

    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
    
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset    
    #pdf.affiche_PDF_avec_contours(offset = -0.1)
    
    # Affichage et sauvegarde des meilleurs résultats
    pdf.affiche_PDF_avec_contours(offset = -0.1, view=[15.6, -19], affiche_tickmarks=False, nom_figure=nom_figure)
    
    

In [5]:
    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [6., 3.]
    mu[1,:] = [4., 6.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [.5, .5]
    sigma[1,:] = [1., 1.5]

    angle = np.array([0., -45.]) 
    prob_C = np.array([0.3, 0.7]) 


    nom_figure = "Figure_20(2)_41.svg"

    
    
    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
    
   
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset    
    #pdf.affiche_PDF_avec_contours(offset = -0.15)
    
    # Affichage et sauvegarde des meilleurs résultats
    pdf.affiche_PDF_avec_contours(offset = -0.15, view=[22., -14], nom_figure=nom_figure)

In [6]:
    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [6., 4.]
    mu[1,:] = [5., 7.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [2., 0.5]
    sigma[1,:] = [1.5, 0.5]

    angle = np.array([-60., 45.]) 
    prob_C = np.array([0.7, 0.5]) 
    

    nom_figure = "Figure_21(1)_41.svg"


    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
    
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset      
    #pdf.affiche_PDF_avec_contours(offset = -0.1)
    
    # Affichage et sauvegarde des meilleurs résultats
    pdf.affiche_PDF_avec_contours(offset = -0.1, view=[21, -18], nom_figure=nom_figure)

In [7]:
    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [5., 5.]
    mu[1,:] = [5., 5.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [2, 2]
    sigma[1,:] = [0.6, 0.6]

    angle = np.array([0., 0.]) 
    prob_C = np.array([0.97, 0.03]) 

    
    nom_figure = "Figure_21(2)_41_qda.svg"


    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
       
    
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset    
    #pdf.affiche_PDF_avec_contours(offset = -0.05)
    
    # Affichage et sauvegarde des meilleurs résultats
    pdf.affiche_PDF_avec_contours(offset = -0.05, view=[28, -21], nom_figure=nom_figure)